In [1]:
import geehydro
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
import os
from os.path import join, basename
import numpy as np
import pandas as pd
#import rioxarray
import matplotlib.pyplot as plt
import rasterio as rio
#from osgeo import gdal
from rasterio.windows import Window , from_bounds, crop
import folium
import pyproj

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWiHrEnvBPJ0cKiJ6XmZZr_hyr3IPySZtQRA8XXhgRv1w7ghw79zcAQ

Successfully saved authorization token.


In [3]:
home_dir = '/home/zacharykeskinen/Documents/uavsar/'
data_dir = join(home_dir,'data')
url_dir = join(data_dir, 'urls')
imgs_dir = join(home_dir, 'imgs')
loc_dir = (join(imgs_dir, os.listdir(imgs_dir)[0]))

class Image:
    def __init__(self, fp, dtype, pol, real_img):
        self.fp = fp
        self.dtype = dtype
        self.pol = pol
        self.comp = real_img
    array = None
    top = None
    bottom = None
    left = None
    right = None
    crs = None
    transform = None
    window = None
    nodata = None

img_list = []
        
for img in os.listdir(loc_dir):
    if 'csv' in img:
        desc = pd.read_csv(join(loc_dir, img))
    if 'tif' in img:
        if len(basename(img).split('.')) == 3:
            fp = join(loc_dir, img)
            real_img = 'real'
            dtype = basename(img).split('.')[-2]
            pol = np.nan
            img_list.append(Image(fp, dtype, pol, real_img))
        else:
            fp = join(loc_dir, img)
            real_img = basename(img).split('.')[-2]
            dtype = basename(img).split('.')[-3]
            pol = basename(img).split('.')[0].split('_')[-2][4:]
            img_list.append(Image(fp, dtype, pol, real_img))

In [4]:
for col in desc.columns:
    print(col)
    print(desc.loc[0,col])
    print('-----------------')

Unnamed: 0
value
-----------------
uavsar rpi annotation file version number
2.3
-----------------
site description
Lowman, CO
-----------------
flight plan comments of pass 1
nan
-----------------
flight plan comments of pass 2
nan
-----------------
processing comments
nan
-----------------
url
http://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName
-----------------
slant range interferogram
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.int
-----------------
slant range unwrapped phase
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.unw
-----------------
slant range correlation
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.cor
-----------------
slant range amplitude of pass 1
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.amp1
-----------------
slant range amplitude of pass 2
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.amp2
-----------------
ground range interferogram
lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.int.grd
-----------------
ground

In [5]:
from datetime import datetime
start_date = pd.to_datetime(desc['start time of acquisition for pass 1'][0])
end_date = pd.to_datetime(desc['start time of acquisition for pass 2'][0])
start_date = f'{start_date.year}-{start_date.month}-{start_date.day}'
end_date = f'{end_date.year}-{end_date.month}-{end_date.day}'
print(start_date)

2021-3-16


In [6]:
left = -115.239170
right = -115.188873
top = 44.328928
bottom = 44.305592

In [7]:
Map = folium.Map(location=[bottom,left],
                   tiles='Stamen Terrain', zoom_start = 13)

site = ee.Geometry.Polygon(
    [[[right, bottom],
      [left, bottom],
      [left, top],
      [right, top]]]);

ic = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3").filterBounds(site).filterMetadata('year','equals',2015).select('tree_canopy_cover')
tree = ic.mean().reproject(crs = ee.Projection('EPSG:4326'), scale=10)
arr = np.array(tree.sampleRectangle(region = site).get('tree_canopy_cover').getInfo())

ic_lc = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global").filterBounds(site).filterMetadata('system:index','equals','2019').select('discrete_classification');
assert ic_lc.size().getInfo() == 1
lc = ic_lc.first()

lf = ee.Image("CSP/ERGo/1_0/US/landforms")

lith = ee.Image("CSP/ERGo/1_0/US/lithology")

start_date = pd.to_datetime(desc['start time of acquisition for pass 1'][0])
end_date = pd.to_datetime(desc['start time of acquisition for pass 2'][0])
start_date = f'{start_date.year}-{start_date.month}-{start_date.day}'
end_date = f'{end_date.year}-{end_date.month}-{end_date.day}'

prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').filter(ee.Filter.date(start_date, end_date));
total_ppt = prism.select('ppt').sum()
mean_temp = prism.select('tmean').mean()

def freezing_days(image):
    return image.updateMask(image.gt(0))

days_mean_af = prism.select('tmean').map(freezing_days).reduce(ee.Reducer.count())
days_max_af = prism.select('tmax').map(freezing_days).reduce(ee.Reducer.count())


with rio.open(img_list[1].fp) as src:
    boundary = src.bounds
    crs = src.crs
    transform_affine = src.transform
    
    my_proj = pyproj.Proj(str(crs).lower())#your data crs

    left_bottom = my_proj(left,bottom, inverse=False)
    right_top = my_proj(right,top, inverse=False)
    win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)
    bounds = rio.windows.bounds(win,src.transform)
    img = src.read(1, window=win)
    nodata = src.nodata

# Overlay raster called img using add_child() function (opacity and bounding box set)
Map.add_child(folium.raster_layers.ImageOverlay(img, opacity=1, 
                                 bounds =[[bottom, left], [top, right]]))

# To see a google satellite view as a basemap
Map.setOptions('HYBRID')

Map.addLayer(tree.clip(site), {'min':0,'max':50}, 'tree coverage', 0)
Map.addLayer(lc.clip(site), {'min':0,'max':200}, 'ground coverage', 0)
Map.addLayer(lf.clip(site), {'min':20, 'max':50}, 'NED land forms', 0)
Map.addLayer(lith.clip(site), {'min':5,'max':30}, 'Lithography', 0)
Map.addLayer(total_ppt.clip(site), {'min':0, 'max':50}, 'Total Precip')
Map.addLayer(days_mean_af.clip(site), {'min':0, 'max':6}, 'Days Mean Above Freezing')
Map.addLayer(days_max_af.clip(site), {'min':0, 'max':6}, 'Days Max Above Freezing')

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

# Display map 
Map

In [8]:
cor_list = [x for x in img_list if x.dtype == 'cor']
with rio.open(cor_list[0].fp) as src:
    
    my_proj = pyproj.Proj(str(src.crs).lower())#your data crs
    left_bottom = my_proj(left,bottom, inverse=False)
    right_top = my_proj(right,top, inverse=False)
    win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)
    target_size = src.read(1, window=win).shape

In [9]:
from scipy import interpolate
def array_resize(arr, target_shape, kind = 'linear'):

    H, W = arr.shape[:2]
    new_H, new_W = target_shape[:2]
    xrange = lambda x: np.linspace(0, 1, x)

    f = interpolate.interp2d(xrange(W), xrange(H), arr, kind=kind)
    new_arr = f(xrange(new_W), xrange(new_H))
    return new_arr

In [10]:
print(days_max_af.sampleRectangle(region = site).get('tmax_count').getInfo())

[[6]]


In [11]:
layer_dic = {tree:'tree_canopy_cover',lc:'discrete_classification', lf:'constant', lith:'b1'}
dtype_dic = {lf:'land_type',lc:'land_cover',tree:'tree',lith:'lithography'}
for im in [tree, lc, lf, lith]:
    fp = None
    real_img = 'real'
    dtype = dtype_dic[im]
    pol = None
    arr = np.array(im.sampleRectangle(region = site).get(layer_dic[im]).getInfo())
    new_arr = array_resize(arr, target_size)
    img = Image(fp, dtype, pol, real_img)
    img.array = new_arr
    img_list.append(img)

In [12]:
left = -115.239170
right = -115.188873
top = 44.328928
bottom = 44.305592

for im in img_list:
    if im.fp == None:
        pass
    else:
        with rio.open(im.fp) as src:
            im.crs = src.crs
            im.transform = src.transform
            im.left = left
            im.right = left
            im.top = top
            im.bottom = bottom

            my_proj = pyproj.Proj(str(src.crs).lower())#your data crs
            left_bottom = my_proj(left,bottom, inverse=False)
            right_top = my_proj(right,top, inverse=False)

            win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)
            im.window = win
            arr = src.read(1, window=win)
            new_arr = array_resize(arr, target_size)
            im.array = new_arr
            im.nodata = src.nodata

In [13]:
import xarray as xr

In [14]:
sub = xr.Dataset()
for im in img_list:
    if im.pol == None:
        sub[f'{im.dtype}'] = (('x','y' ), im.array)
    else:
        sub[f'{im.dtype}_{im.pol}_{im.comp}'] = (('x','y' ), im.array)

In [15]:
for col in desc.columns:
    if col not in desc.columns[0]:
        if desc.loc[1,col] != '&':
            sub.attrs[col] =  str(desc.loc[0,col])+ ' ' +str(desc.loc[1,col])
        else:    
            sub.attrs[col] =  str(desc.loc[0,col])

In [16]:
left = -115.239170
right = -115.188873
top = 44.328928
bottom = 44.305592
sub.attrs['subset_coords'] = [left,bottom,right,top]

In [17]:
sub

<xarray.Dataset>
Dimensions:           (x: 533, y: 787)
Dimensions without coordinates: x, y
Data variables: (12/33)
    hgt_HH_real       (x, y) float64 2.125e+03 2.126e+03 ... 2.623e+03 2.624e+03
    int_VV_imaginary  (x, y) float64 0.01291 0.0104 0.02355 ... 0.01009 0.003904
    inc_nan_real      (x, y) float64 1.116 1.117 1.117 ... 1.342 1.339 1.337
    int_HV_real       (x, y) float64 -0.005895 -0.004708 ... -0.0008028 0.001729
    int_HH_imaginary  (x, y) float64 0.03383 0.02588 0.02119 ... 0.02457 0.01235
    amp2_HH_real      (x, y) float64 0.164 0.2162 0.2913 ... 0.2012 0.1786
    ...                ...
    cor_HV_real       (x, y) float64 0.36 0.3356 0.3162 ... 0.3999 0.3257 0.1749
    hgt_VH_real       (x, y) float64 2.125e+03 2.126e+03 ... 2.623e+03 2.624e+03
    tree              (x, y) float64 39.0 39.0 39.0 39.0 ... 4.0 4.0 4.0 4.0
    land_cover        (x, y) float64 111.0 111.0 111.0 ... 126.0 126.0 126.0
    land_type         (x, y) float64 41.0 41.0 41.0 41.0 ... 11.42 11.0 11.0
    lithography       (x, y) float64 19.0 19.0 19.0 19.0 ... 5.0 5.0 5.0 5.0
Attributes: (12/235)
    uavsar rpi annotation file version number:       2.3 -
    site description:                                Lowman, CO
    flight plan comments of pass 1:                  nan
    flight plan comments of pass 2:                  nan
    processing comments:                             nan
    url:                                             http://uavsar.jpl.nasa.g...
    ...                                              ...
    slt_phs.dsp_cmap:                                cmy
    grd_phs.dsp_cmap:                                cmy
    slc_phs.dsp_cmap:                                cmy
    set_hddr:                                        0 bytes
    set_tail:                                        0 bytes
    subset_coords:                                   [-115.23917, 44.305592, ...

In [18]:
sub.to_netcdf('../data/subsets/banner_lowman_05208_21019-019_21021-007_0006d.nc')